## Pipeline for Data Frame Creation and Cleaning

### 1. Load Necessary Packages

In [3]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from io import StringIO
import zipfile
import shutil
from io import BytesIO
import base64
import json
import requests
import time
from zipfile import ZipFile
from datetime import datetime as dt
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed


### 2. Download Data

#### ISO-NE

In [ ]:
def download_isone_consolidated_lmp_by_quarter(start_date: str, end_date: str, save_path: str = 'iso_data/isone_lmp_data') -> None:
    """
    Downloads ISO-NE Day-Ahead LMP CSVs for each day in the date range,
    cleans and combines the data, and saves one CSV per quarter.

    No individual daily files are written to disk.
    """

    os.makedirs(save_path, exist_ok=True)
    all_data = []

    # Parse date range
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')

    current_dt = start_dt
    while current_dt <= end_dt:
        date_str = current_dt.strftime('%Y%m%d')
        filename = f"WW_DALMP_ISO_{date_str}.csv"
        url = f"https://www.iso-ne.com/static-transform/csv/histRpts/da-lmp/{filename}"

        print(f"📥 Downloading {filename}...")

        try:
            response = requests.get(url)
            response.raise_for_status()

            # Read directly from the response content
            df = pd.read_csv(StringIO(response.text), skiprows=4, header=0)
            df = df.iloc[1:].reset_index(drop=True)  # Remove type row

            df['Fetched_Date'] = current_dt.strftime('%Y-%m-%d')
            all_data.append(df)

        except requests.exceptions.HTTPError:
            print(f"❌ File not found for {date_str} — skipping.")
        except Exception as e:
            print(f"⚠️ Error processing {date_str}: {e}")

        current_dt += timedelta(days=1)

    if not all_data:
        print("🚫 No data was downloaded.")
        return

    # Combine all data
    full_df = pd.concat(all_data, ignore_index=True)

    # Parse the actual 'Date' column
    full_df['Date'] = pd.to_datetime(full_df['Date'], errors='coerce')
    full_df = full_df.dropna(subset=['Date'])

    # Assign each row to a calendar quarter
    full_df['Quarter'] = full_df['Date'].dt.to_period('Q')

    # Save one CSV per quarter
    for quarter, group in full_df.groupby('Quarter'):
        quarter_str = str(quarter).replace('/', '')  # e.g., '2024Q1'
        output_filename = f"isone_lmp_{quarter_str}.csv"
        output_path = os.path.join(save_path, output_filename)

        group.drop(columns='Quarter').to_csv(output_path, index=False)
        print(f"✅ Saved {output_filename} with {len(group)} records.")

if __name__ == "__main__":
    download_isone_consolidated_lmp_by_quarter("2022-01-01", "2024-12-31")

#### NYISO

In [ ]:
# NYISO Zonal Data
def download_nyiso_zone_lmp_monthly_by_quarter(start_year: int, end_year: int, save_path='iso_data/nyiso_data/nyiso_zonal_lmp_data'):
    """
    Downloads NYISO monthly ZONAL LBMP ZIPs, extracts daily CSVs, aggregates by quarter,
    saves quarterly CSVs, and deletes the extracted folder afterward.
    """

    os.makedirs(save_path, exist_ok=True)
    temp_extract_path = os.path.join(save_path, 'extracted')
    os.makedirs(temp_extract_path, exist_ok=True)
    all_data = []

    # Generate list of months
    months = pd.date_range(start=f'{start_year}-01-01', end=f'{end_year}-12-31', freq='MS')

    for dt in months:
        yyyymmdd = dt.strftime('%Y%m01')
        zip_filename = f"{yyyymmdd}damlbmp_zone_csv.zip"  # ⬅️ _zone_ here!
        url = f"https://mis.nyiso.com/public/csv/damlbmp/{zip_filename}"

        print(f"⬇️ Downloading {zip_filename}...")

        try:
            response = requests.get(url)
            if response.status_code != 200:
                print(f"❌ File not found for {dt.strftime('%B %Y')} — skipping.")
                continue

            # Extract ZIP in memory
            with zipfile.ZipFile(BytesIO(response.content)) as z:
                z.extractall(temp_extract_path)
            print(f"✅ Extracted {zip_filename}")

        except Exception as e:
            print(f"⚠️ Error processing {zip_filename}: {e}")

    # Collect all extracted CSVs
    all_files = [os.path.join(temp_extract_path, f) for f in os.listdir(temp_extract_path) if f.endswith('.csv')]
    print(f"📂 Found {len(all_files)} daily CSVs to process.")

    for file in all_files:
        try:
            df = pd.read_csv(file)
            all_data.append(df)
        except Exception as e:
            print(f"⚠️ Skipping {file}: {e}")

    if not all_data:
        print("No data was processed.")
        return

    # Combine
    full_df = pd.concat(all_data, ignore_index=True)

    # Detect timestamp column
    timestamp_col = next((col for col in full_df.columns if 'Time' in col or 'time' in col.lower()), None)
    if not timestamp_col:
        raise ValueError("Couldn't find a timestamp column in the combined CSVs.")

    full_df['Date'] = pd.to_datetime(full_df[timestamp_col], errors='coerce')
    full_df = full_df.dropna(subset=['Date'])

    # Assign calendar quarter
    full_df['Quarter'] = full_df['Date'].dt.to_period('Q')

    # Save one CSV per quarter
    for quarter, group in full_df.groupby('Quarter'):
        quarter_str = str(quarter).replace('/', '')
        output_path = os.path.join(save_path, f'nyiso_zonal_lmp_{quarter_str}.csv')
        group.drop(columns='Quarter').to_csv(output_path, index=False)
        print(f"✅ Saved {output_path} with {len(group)} rows.")

    # ✅ Clean up extracted folder
    print(f"🧹 Removing extracted folder: {temp_extract_path}")
    shutil.rmtree(temp_extract_path)
    print("✅ Cleanup complete.")

# Example usage
if __name__ == "__main__":
    download_nyiso_zone_lmp_monthly_by_quarter(start_year=2022, end_year=2024)

# NYISO Generator Data
def download_nyiso_gen_lmp_monthly_by_quarter(start_year: int, end_year: int, save_path='iso_data/nyiso_data/nyiso_gen_lmp_data'):
    """
    Downloads NYISO monthly generator LBMP ZIPs, extracts daily CSVs, aggregates by quarter,
    saves quarterly CSVs, and deletes the extracted folder afterward.
    """

    os.makedirs(save_path, exist_ok=True)
    temp_extract_path = os.path.join(save_path, 'extracted')
    os.makedirs(temp_extract_path, exist_ok=True)
    all_data = []

    # Generate list of months
    months = pd.date_range(start=f'{start_year}-01-01', end=f'{end_year}-12-31', freq='MS')

    for dt in months:
        yyyymmdd = dt.strftime('%Y%m01')
        zip_filename = f"{yyyymmdd}damlbmp_gen_csv.zip"
        url = f"https://mis.nyiso.com/public/csv/damlbmp/{zip_filename}"

        print(f"⬇️ Downloading {zip_filename}...")

        try:
            response = requests.get(url)
            if response.status_code != 200:
                print(f"❌ File not found for {dt.strftime('%B %Y')} — skipping.")
                continue

            # Extract ZIP in memory
            with zipfile.ZipFile(BytesIO(response.content)) as z:
                z.extractall(temp_extract_path)
            print(f"✅ Extracted {zip_filename}")

        except Exception as e:
            print(f"⚠️ Error processing {zip_filename}: {e}")

    # Collect all extracted CSVs
    all_files = [os.path.join(temp_extract_path, f) for f in os.listdir(temp_extract_path) if f.endswith('.csv')]
    print(f"📂 Found {len(all_files)} daily CSVs to process.")

    for file in all_files:
        try:
            df = pd.read_csv(file)
            all_data.append(df)
        except Exception as e:
            print(f"⚠️ Skipping {file}: {e}")

    if not all_data:
        print("No data was processed.")
        return

    # Combine
    full_df = pd.concat(all_data, ignore_index=True)

    # Detect timestamp column
    timestamp_col = next((col for col in full_df.columns if 'Time' in col or 'time' in col.lower()), None)
    if not timestamp_col:
        raise ValueError("Couldn't find a timestamp column in the combined CSVs.")

    full_df['Date'] = pd.to_datetime(full_df[timestamp_col], errors='coerce')
    full_df = full_df.dropna(subset=['Date'])

    # Assign calendar quarter
    full_df['Quarter'] = full_df['Date'].dt.to_period('Q')

    # Save one CSV per quarter
    for quarter, group in full_df.groupby('Quarter'):
        quarter_str = str(quarter).replace('/', '')
        output_path = os.path.join(save_path, f'nyiso_lmp_{quarter_str}.csv')
        group.drop(columns='Quarter').to_csv(output_path, index=False)
        print(f"✅ Saved {output_path} with {len(group)} rows.")

    # ✅ Clean up extracted folder
    print(f"🧹 Removing extracted folder: {temp_extract_path}")
    shutil.rmtree(temp_extract_path)
    print("✅ Cleanup complete.")

# Example usage
if __name__ == "__main__":
    download_nyiso_gen_lmp_monthly_by_quarter(start_year=2022, end_year=2023)

# combine zonal and generator data

# Paths to your folders
nodal_dir = 'iso_data/nyiso_data/nyiso_gen_lmp_data'
zonal_dir = 'iso_data/nyiso_data/nyiso_zonal_lmp_data'
output_dir = 'iso_data/nyiso_data/nyiso_combined_quarters'
os.makedirs(output_dir, exist_ok=True)

# Get all quarter identifiers from filenames (e.g., '2022Q1')
nodal_quarters = [f.replace('nyiso_lmp_', '').replace('.csv', '') for f in os.listdir(nodal_dir) if f.endswith('.csv')]
zonal_quarters = [f.replace('nyiso_zonal_lmp_', '').replace('.csv', '') for f in os.listdir(zonal_dir) if f.endswith('.csv')]

# Get common quarters that exist in both folders
all_quarters = sorted(set(nodal_quarters) & set(zonal_quarters))

# Process and merge each quarter
for quarter in all_quarters:
    nodal_file = os.path.join(nodal_dir, f'nyiso_lmp_{quarter}.csv')
    zonal_file = os.path.join(zonal_dir, f'nyiso_zonal_lmp_{quarter}.csv')
    
    # Read files and tag node_type
    nodal_df = pd.read_csv(nodal_file)
    nodal_df['node_type'] = 'nodal'
    
    zonal_df = pd.read_csv(zonal_file)
    zonal_df['node_type'] = 'zonal'
    
    # Combine and save
    combined_df = pd.concat([nodal_df, zonal_df], ignore_index=True)
    output_path = os.path.join(output_dir, f'nyiso_combined_{quarter}.csv')
    combined_df.to_csv(output_path, index=False)
    
    print(f"✅ Combined and saved {output_path}")


#### PJM

#### MISO

In [ ]:
# for 2023 to present data extraction
def download_and_extract_zip(url):
    """
    Downloads and extracts all CSVs from a MISO ZIP archive.
    Returns a list of DataFrames.
    """
    print(f"  Trying: {url}")
    response = requests.get(url)

    if response.status_code != 200:
        return []

    if response.content[:2] != b'PK':  # ZIP files start with 'PK'
        return []

    dfs = []
    try:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            for filename in z.namelist():
                if filename.endswith(".csv"):
                    print(f"    Extracting: {filename}")
                    with z.open(filename) as f:
                        try:
                            df = pd.read_csv(f, low_memory=False)
                            df["SOURCE_FILE"] = filename
                            dfs.append(df)
                        except Exception as e:
                            print(f"    Failed to parse {filename}: {e}")
    except zipfile.BadZipFile:
        print(f"    Error: File at {url} is not a valid ZIP archive.")
    return dfs

def scrape_miso_quarterly_zips(year, quarters, output_dir="iso_data/miso_data"):
    """
    Downloads and processes MISO LMP ZIP files for given year and quarters,
    handling naming inconsistencies and saving each quarter separately.
    """
    os.makedirs(output_dir, exist_ok=True)

    quarter_map_variants = {
        "Q1": ["Jan-Mar", "Jan_Mar"],
        "Q2": ["Apr-Jun", "Apr_Jun"],
        "Q3": ["Jul-Sep", "Jul_Sep"],
        "Q4": ["Oct-Dec", "Oct_Dec"]
    }
    suffixes = ["DA_LMPs.zip", "DA_LMP.zip"]
    year_sep_variants = ["_", "-"]

    for q in quarters:
        print(f"\n📦 Processing {year} {q}")
        success = False
        for quarter_str in quarter_map_variants[q]:
             for sep in year_sep_variants:
                for suffix in suffixes:
                    filename = f"{year}{sep}{quarter_str}_{suffix}"
                    url = f"https://docs.misoenergy.org/marketreports/{filename}"

                    dfs = download_and_extract_zip(url)
                    if dfs:
                        combined_df = pd.concat(dfs, ignore_index=True)
                        out_path = os.path.join(output_dir, f"{year}_{q}.csv")
                        combined_df.to_csv(out_path, index=False)
                        print(f"✅ Saved to {out_path}")
                        success = True
                        break  # stop after the first successful variant
                if success:
                    break
                if not success:
                    print(f"⚠️ No valid file found for {year} {q}")

# Example usage
if __name__ == "__main__":
    scrape_miso_quarterly_zips(year=2025, quarters=["Q1", "Q2", "Q3", "Q4"])

# 2022 and earlier
def extract_and_save_quarter_from_nested_zip(nested_zip_bytes, nested_filename, year, output_dir):
    """
    Extracts CSVs from a nested ZIP archive and saves them as a quarterly file
    based on the name of the nested ZIP.
    """
    # Identify quarter from filename
    quarter_hint = ""
    if any(m in nested_filename for m in ["Jan", "Feb", "Mar"]):
        quarter_hint = "Q1"
    elif any(m in nested_filename for m in ["Apr", "May", "Jun"]):
        quarter_hint = "Q2"
    elif any(m in nested_filename for m in ["Jul", "Aug", "Sep"]):
        quarter_hint = "Q3"
    elif any(m in nested_filename for m in ["Oct", "Nov", "Dec"]):
        quarter_hint = "Q4"
    else:
        print(f"⚠️ Could not identify quarter from: {nested_filename}")
        return

    # Extract CSVs from nested ZIP
    dfs = []
    try:
        with zipfile.ZipFile(io.BytesIO(nested_zip_bytes)) as nested_zip:
            for filename in nested_zip.namelist():
                if filename.endswith(".csv"):
                    print(f"    📄 Extracting CSV: {filename}")
                    with nested_zip.open(filename) as f:
                        try:
                            df = pd.read_csv(f, low_memory=False)
                            df["SOURCE_FILE"] = filename
                            dfs.append(df)
                        except Exception as e:
                            print(f"      ⚠️ Failed to read {filename}: {e}")
    except zipfile.BadZipFile:
        print(f"⚠️ Invalid nested ZIP: {nested_filename}")
        return

    # Save only that quarter’s data
    if dfs:
        combined = pd.concat(dfs, ignore_index=True)
        out_path = os.path.join(output_dir, f"{year}_{quarter_hint}.csv")
        combined.to_csv(out_path, index=False)
        print(f"✅ Saved quarterly CSV: {out_path}")
    else:
        print(f"⚠️ No CSVs extracted from: {nested_filename}")

def download_and_process_annual_zip(year, output_dir="miso_data"):
    """
    Downloads a yearly MISO ZIP (which contains nested ZIPs by month/quarter),
    extracts quarterly data, and saves each quarter separately.
    """
    os.makedirs(output_dir, exist_ok=True)
    filename = f"{year}01_DA_LMPs_zip.zip"
    url = f"https://docs.misoenergy.org/marketreports/{filename}"
    print(f"\n📦 Processing annual archive for {year}: {url}")

    try:
        response = requests.get(url)
        if response.status_code != 200 or response.content[:2] != b'PK':
            print("❌ Failed to download or invalid ZIP format.")
            return

        with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
            for nested_name in outer_zip.namelist():
                if nested_name.endswith(".zip"):
                    print(f"  📦 Found nested ZIP: {nested_name}")
                    with outer_zip.open(nested_name) as nested_file:
                        nested_bytes = nested_file.read()
                        extract_and_save_quarter_from_nested_zip(
                            nested_zip_bytes=nested_bytes,
                            nested_filename=nested_name,
                            year=year,
                            output_dir=output_dir
                        )
    except Exception as e:
        print(f"⚠️ Error processing archive: {e}")

if __name__ == "__main__":
    download_and_process_annual_zip(2022, output_dir="iso_data/miso_data")


#### ERCOT

In [ ]:
token_url = "https://ercotb2c.b2clogin.com/ercotb2c.onmicrosoft.com/B2C_1_PUBAPI-ROPC-FLOW/oauth2/v2.0/token"

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "password",
    "scope": "openid fec253ea-0d06-4272-a5e6-b478baeecd70 offline_access",
    "client_id": "fec253ea-0d06-4272-a5e6-b478baeecd70",
    "username": "alanwang2025@u.northwestern.edu",
    "password": "Fork102$"
}

response = requests.post(token_url, data=token_data, headers=headers)
response.raise_for_status()
access_token = response.json()["access_token"]
print("✅ Token received.")

token_parts = access_token.split(".")
payload = token_parts[1] + '=' * (-len(token_parts[1]) % 4)  # fix padding
decoded = json.loads(base64.urlsafe_b64decode(payload.encode()).decode())
print(json.dumps(decoded, indent=2))

access_token

# Replace with your real token and subscription key
subscription_key = "7076e411aeeb461e8bb085df1690f0cd"


# Headers for API access
headers = {
    "Authorization": f"Bearer {access_token}",
    "Ocp-Apim-Subscription-Key": subscription_key
}

# Step 1: Fetch archive metadata
print("📦 Fetching ERCOT DAM LMP archive list...")
endpoint = "https://api.ercot.com/api/public-reports/archive/np4-183-cd"
response = requests.get(endpoint, headers=headers)
response.raise_for_status()
archives = response.json().get("archives", [])
print(f"✅ Found {len(archives)} archive entries.")

# ✅ Filter for only archives from 2022 onwards
archives = [
    archive for archive in archives
    if "postDatetime" in archive and
       dt.fromisoformat(archive["postDatetime"][:19]) >= dt(2022, 1, 1) and
       dt.fromisoformat(archive["postDatetime"][:19]) <= dt(2022, 10, 1)
]

print(f"✅ Found {len(archives)} archive entries from 2022 onward.")

# Output dir
output_dir = "ercot_dam_outputs"
os.makedirs(output_dir, exist_ok=True)

# Step 2: Download and parse
for i, archive in enumerate(archives):
    download_url = f"https://api.ercot.com/api/public-reports/archive/np4-183-cd?download={archive['docId']}"
    print(f"\n⬇️ Downloading ({i+1}): {archive['friendlyName']}")
    
    for _ in range(3):
        r = requests.get(download_url, headers=headers)
        if r.status_code == 429:
            print("⏳ Rate limit hit. Waiting...")
            time.sleep(2)
        else:
            break

    if r.status_code != 200:
        print(f"⚠️ Failed to download. Status: {r.status_code}")
        continue

    content_type = r.headers.get("Content-Type", "")
    raw = r.content

    try:
        if raw.startswith(b'PK'):  # ZIP magic number
            print("📦 Detected ZIP archive")
            with ZipFile(BytesIO(raw)) as z:
                for file_info in z.infolist():
                    if file_info.filename.endswith(".csv"):
                        with z.open(file_info) as f:
                            df = pd.read_csv(f, engine='python', on_bad_lines='skip')
        else:
            print("🧾 Detected raw CSV file")
            decoded = raw.decode("utf-8", errors="ignore")
            df = pd.read_csv(StringIO(decoded), engine='python', on_bad_lines='skip')

        # Normalize column names
        df.columns = df.columns.str.strip().str.lower().str.replace(" ", "")
        print("📑 Columns:", df.columns.tolist())

        # Accept 'deliverydate' or fallback options
        date_col = None
        for col in df.columns:
            if col in ["deliverydate", "delivery_date", "delvdate"]:
                date_col = col
                break

        if not date_col:
            print(f"❌ No recognized delivery date column in: {archive['friendlyName']}")
            continue

        df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
        df["quarter"] = df[date_col].dt.to_period("Q").astype(str)

        for quarter, group in df.groupby("quarter"):
            out_path = os.path.join(output_dir, f"ERCOT_LMP_{quarter}.csv")
            if os.path.exists(out_path):
                group.to_csv(out_path, mode='a', header=False, index=False)
            else:
                group.to_csv(out_path, index=False)
        
        print(f"✅ Saved: {archive['friendlyName']}")
    except Exception as e:
        print(f"❌ Error parsing {archive['friendlyName']}: {e}")

#### SPP

#### CAISO

In [ ]:
def download_day(date, nodes=None):
    """Download and extract DAM_LMP CSV for a single day"""
    base_url = "http://oasis.caiso.com/oasisapi/SingleZip"
    date_str = date.strftime("%Y%m%d")
    
    params = {
        "resultformat": "6",
        "queryname": "PRC_LMP",
        "version": "12",
        "market_run_id": "DAM",
        "startdatetime": f"{date_str}T08:00-0000",
        "enddatetime": f"{date_str}T08:00-0000",
        "grp_type": "ALL"
    }

    try:
        response = requests.get(base_url, params=params, timeout=60)
        response.raise_for_status()
    except Exception as e:
        print(f"❌ Download error for {date_str}: {e}")
        return None

    try:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            for name in z.namelist():
                if "DAM_LMP" in name and name.endswith(".csv"):
                    with z.open(name) as f:
                        df = pd.read_csv(f)
                        if nodes is not None:
                            df = df[df["NODE"].isin(nodes)]
                        return df
    except Exception as e:
        print(f"❌ ZIP processing error for {date_str}: {e}")
        return None

def fetch_caiso_dam_lmp_parallel(start_date, end_date, nodes=None, max_workers=8):
    """Parallel downloader for CAISO DAM_LMP data"""
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    date_list = [start_dt + timedelta(days=i) for i in range((end_dt - start_dt).days + 1)]

    all_data = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(download_day, date, nodes): date for date in date_list}
        for future in tqdm(as_completed(futures), total=len(futures), desc="CAISO Parallel"):
            result = future.result()
            if result is not None:
                all_data.append(result)

    return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

# Example usage
if __name__ == "__main__":
    save_folder = "iso_data/caiso_data"  # ✅ Your desired folder path
    os.makedirs(save_folder, exist_ok=True)   # ✅ Create it if it doesn't exist

    df = fetch_caiso_dam_lmp_parallel(
        start_date="2023-01-01",
        end_date="2024-01-01",
        max_workers=6
    )

    if not df.empty:
        save_path = os.path.join(save_folder, "caiso_dam_lmp_parallel.csv")
        df.to_csv(save_path, index=False)
        print(f"✅ Saved to {save_path}")
    else:
        print("🚫 No data downloaded.")

#### Interconnection Queues

#### EIA Wholesale Electricity Trades

In [ ]:
# Define the base URL and the list of file names
base_url = "https://www.eia.gov/electricity/wholesale/xls/archive/"
file_names = [
    "ice_electric-2022final.xlsx",
    "ice_electric-2023final.xlsx",
    "ice_electric-2024final.xlsx"
]

# Directory where the files will be saved
save_dir = 'iso_data/Wholesale_Pricing_Data'

# Loop to download each file
for file_name in file_names:
    full_url = base_url + file_name
    response = requests.get(full_url)

    if response.status_code == 200:
        file_path = os.path.join(save_dir, file_name)
        with open(file_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded: {file_name}")
    else:
        print(f"Failed to download: {file_name} (Status code: {response.status_code})")

### 3. Organize Each Data Source Into Yearly Format

In [ ]:
def combine_quarters(folder, year=None):
    """
    Combines quarterly CSVs in a folder.
    If 'year' is provided, only combines files containing that year in the filename.
    """
    files = sorted([f for f in os.listdir(folder) if f.endswith('.csv')])

    if year is not None:
        files = [f for f in files if str(year) in f]

    dfs = []

    for f in files:
        path = os.path.join(folder, f)
        try:
            df = pd.read_csv(path, low_memory=False)
            dfs.append(df)
            print(f"✅ Loaded {f}")
        except Exception as e:
            print(f"⚠️ Failed to load {f}: {e}")

    if not dfs:
        print(f"⚠️ No files found for year {year} in {folder}")
        return pd.DataFrame()

    combined = pd.concat(dfs, ignore_index=True)
    print(f"🧩 Combined {len(dfs)} files with {len(combined):,} rows for year {year}.")
    return combined

# Combine only MISO yearly files
miso_2023_df = combine_quarters('iso_data/miso_data', year=2023)

# Combine only ISO-NE yearly files
isone_2023_df = combine_quarters('iso_data/isone_lmp_data', year=2023)

# Combine only NYISO yearly files
nyo_2023_df = combine_quarters('iso_data/nyiso_data/nyiso_combined_quarters', year=2023)

# Combine only ERCOT yearly files
ercot_2023_df = combine_quarters('iso_data/ercot_dam_outputs', year=2023)

# Combine only SPP yearly files
spp_2023_df = combine_quarters('iso_data/spp_lmp_quarters', year=2023)

# Combine only PJM 2023 files
pjm_2023_df = combine_quarters('iso_data/pjm_data', year=2023)

# CAISO already downloaded into yearly format

### 4. Combine Data Frames

#### ISO-NE, NYISO, PJM + Respective Interconnection Queues and Wholesale Electricity Trades

In [ ]:
# Combine NE/Mid-atlantic ISO dataframes
combined_ne_raw_df = pd.concat([
    nyo_2023_df,
    isone_2023_df, 
    pjm_2023_df
], ignore_index=True)

print(combined_ne_raw_df.head())

print(f"✅ Combined raw DataFrame shape: {combined_ne_raw_df.shape}")

def clean_pjm(pjm_df):
    """Clean PJM dataframe to standard format."""
    pjm_df = pjm_df.copy()
    pjm_df['timestamp_utc'] = pd.to_datetime(pjm_df['datetime_beginning_utc'], utc = True)
    pjm_df['iso'] = 'PJM'
    pjm_df['Location Name'] = pjm_df['pnode_name']
    pjm_df['Location Type'] = 'Node'
    pjm_df['LMP'] = pjm_df['total_lmp_da']  # or 'total_lmp_rt' if you prefer real-time
    pjm_df['MCC'] = pjm_df['congestion_price_da']
    pjm_df['MLC'] = pjm_df['marginal_loss_price_da']
    
    return pjm_df[['timestamp_utc', 'iso', 'Location Name', 'Location Type', 'LMP', 'MCC', 'MLC']]

def clean_nyiso(nyiso_df):
    """Clean NYISO dataframe to standard format."""
    nyiso_df = nyiso_df.copy()
    nyiso_df['timestamp_utc'] = pd.to_datetime(nyiso_df['Date'], utc = True)
    nyiso_df['iso'] = 'NYISO'
    nyiso_df['Location Name'] = nyiso_df['Name']
    nyiso_df['Location Type'] = 'Node'
    nyiso_df['LMP'] = nyiso_df['LBMP ($/MWHr)'] 
    nyiso_df['MCC'] = nyiso_df['Marginal Cost Congestion ($/MWHr)']
    nyiso_df['MLC'] = nyiso_df['Marginal Cost Losses ($/MWHr)']
    
    return nyiso_df[['timestamp_utc', 'iso', 'Location Name', 'Location Type', 'LMP', 'MCC', 'MLC']]

def clean_isone(isone_df):
    """Clean ISO-NE dataframe to standard format."""
    isone_df = isone_df.copy()

    # Safe handling of 'Hour Ending'
    isone_df['Hour Ending'] = isone_df['Hour Ending'].astype(str)
    isone_df = isone_df[isone_df['Hour Ending'].str.isnumeric()]
    isone_df['Hour Ending'] = isone_df['Hour Ending'].astype(int)
    isone_df['timestamp_utc'] = pd.to_datetime(isone_df['Date']) + pd.to_timedelta(isone_df['Hour Ending'] - 1, unit='h')
    isone_df['timestamp_utc'] = pd.to_datetime(isone_df['timestamp_utc'], utc=True)

    isone_df['iso'] = 'ISO-NE'
    isone_df['Location Name'] = isone_df['Location Name']
    isone_df['Location Type'] = 'Node'  # Simplify network node to Node
    isone_df['LMP'] = isone_df['Locational Marginal Price']
    isone_df['MCC'] = isone_df['Congestion Component']
    isone_df['MLC'] = isone_df['Marginal Loss Component']
    
    return isone_df[['timestamp_utc', 'iso', 'Location Name', 'Location Type', 'LMP', 'MCC', 'MLC']]

def combine_isos(pjm_df, nyiso_df, isone_df):
    """Combine cleaned ISO dataframes into one."""
    pjm_clean = clean_pjm(pjm_df)
    nyiso_clean = clean_nyiso(nyiso_df)
    isone_clean = clean_isone(isone_df)
    
    combined_df = pd.concat([pjm_clean, nyiso_clean, isone_clean], ignore_index=True)
    return combined_df

combined_ne_df = combine_isos(pjm_2023_df, nyo_2023_df, isone_2023_df)

combined_ne_df = combined_ne_df.sort_values('timestamp_utc').reset_index(drop=True)

# 🧠 Now set timestamp_utc as index
combined_ne_df = combined_ne_df.set_index('timestamp_utc')
combined_ne_df

combined_ne_df_2023 = combined_ne_df[['iso', 'Location Name', 'Location Type', 'LMP', 'MCC', 'MLC']]
combined_ne_df_2023['node'] = 'node'
combined_ne_df_2023

output_path = 'iso_data/yearly_combined_data/combined_ne_2023_lmp_data.csv'
combined_ne_df_2023.to_csv(output_path)

print(f"✅ Successfully saved to {output_path}")

In [ ]:
# add in interconnection queue data
queues = pd.read_excel('iso_data/queues_2024.xlsx', sheet_name=1)

def standardize_dates(series):
    return pd.to_datetime(series, 
                          infer_datetime_format=True,
                          errors='coerce')

queues['q_date'] = standardize_dates(queues['q_date'])
queues['ia_date'] = standardize_dates(queues['ia_date'])
queues['wd_date'] = standardize_dates(queues['wd_date'])
queues['on_date'] = standardize_dates(queues['on_date'])

cutoff = pd.Timestamp('2022-01-01')
filtered_queues = queues.loc[(queues['q_date'] >= cutoff) & 
                             ((queues['ia_date'] >= cutoff)|(queues['ia_date'].isnull())) & 
                             ((queues['wd_date'] >= cutoff)|(queues['wd_date'].isnull())) &
                             ((queues['on_date'] >= cutoff)|(queues['on_date'].isnull()))]

# q_date: date when project entered queue
# ia_date: date of signed interconnection agreement
# wd_date: date project withdrawn from queue
# on_date: date project became operational

selected_cols = ['q_id', 'region', 'q_date', 
                 'ia_date', 'wd_date', 'on_date']

selected_regions = ['PJM', 'ERCOT', 'CAISO', 'SPP', 'NYISO', 'ISO-NE']

filtered_queues = filtered_queues[filtered_queues['region'].isin(selected_regions)].reset_index(drop=True)

def queue_duration(df, start, end):
    difference = df[end] - df[start]
    return difference.dt.days

filtered_queues['days_to_ia'] = queue_duration(filtered_queues, 'q_date', 'ia_date')
filtered_queues['days_to_wd'] = queue_duration(filtered_queues, 'q_date', 'wd_date')
filtered_queues['days_to_on'] = queue_duration(filtered_queues, 'q_date', 'on_date')

as_of = pd.to_datetime('2023-12-31')
pending = filtered_queues[['ia_date','wd_date','on_date']].isna().all(axis=1)

filtered_queues['days_pending'] = np.where(
    pending,
    (as_of - filtered_queues['q_date']).dt.days,
    np.nan)

selected_cols = selected_cols + ['days_to_ia', 'days_to_wd', 'days_to_on', 'days_pending']

# 1) build a “long” events frame: one row per event occurrence
events = []
for event, date_col, days_col in [
    ('ia', 'ia_date', 'days_to_ia'),
    ('wd', 'wd_date', 'days_to_wd'),
    ('on', 'on_date', 'days_to_on'),
]:
    tmp = (
        filtered_queues
        .loc[filtered_queues[date_col].notna(), ['region', date_col, days_col]]
        .rename(columns={date_col: 'date', days_col: 'days_to'})
    )
    tmp['event'] = event
    events.append(tmp)
events = pd.concat(events, ignore_index=True)

events.sort_values(by=['region','date'])

# 1) pivot out daily counts and daily average days_to
daily_counts = (
    events
      .groupby(['region','date','event'])
      .size()
      .unstack('event', fill_value=0)
      .rename(columns={'ia':'count_ia','wd':'count_wd','on':'count_on'}))

daily_days = (
    events
      .groupby(['region','date','event'])['days_to']
      .mean()
      .unstack('event')
      .rename(columns={'ia':'days_to_ia','wd':'days_to_wd','on':'days_to_on'}))

daily = (
    daily_counts
      .join(daily_days, how='outer')
      .sort_index())

# 2) reindex to every calendar date so cumsum/expanding works
all_dates = pd.date_range(
    events['date'].min().floor('D'),
    events['date'].max().ceil('D'),
    freq='D'
)
idx = pd.MultiIndex.from_product(
    [daily.index.levels[0], all_dates],
    names=['region','date']
)
daily = daily.reindex(idx, fill_value=0).sort_index()

# 3) cumulative sums of counts
daily['cum_ia'] = daily.groupby(level='region')['count_ia'].cumsum()
daily['cum_wd'] = daily.groupby(level='region')['count_wd'].cumsum()
daily['cum_on'] = daily.groupby(level='region')['count_on'].cumsum()

# 4) cumulative sums & counts of days_to, then expanding mean
for ev in ['ia','wd','on']:
    # running sum of days_to
    daily[f'sum_days_to_{ev}'] = (
        daily[f'days_to_{ev}']
          .groupby(level='region')
          .cumsum()
    )
    # running count of events (same as cum_count)
    daily[f'cnt_days_to_{ev}'] = daily[f'count_{ev}'].groupby(level='region').cumsum()
    # expanding average = sum / count
    daily[f'avg_days_to_{ev}'] = (
        daily[f'sum_days_to_{ev}'] / daily[f'cnt_days_to_{ev}'])

# 5) clean up: drop intermediate columns
final = daily.reset_index().drop(
    columns=[f'days_to_{ev}'      for ev in ['ia','wd','on']]
           +[f'sum_days_to_{ev}'  for ev in ['ia','wd','on']]
           +[f'cnt_days_to_{ev}'  for ev in ['ia','wd','on']])

# 0) your cutoff
as_of = pd.to_datetime('2023-12-31')

# 1) build the calendar of snapshot dates **only** through the cutoff
all_dates = pd.date_range(
    start=filtered_queues['q_date'].min().floor('D'),
    end=as_of,
    freq='D')

# 2) pull in each project’s dates
proj_raw = filtered_queues[['region','q_date','ia_date','wd_date','on_date']].copy()

# 3) Cartesian-merge so each project is paired with each snapshot date
dates_df = pd.DataFrame({'date': all_dates})
projects = (
    proj_raw
      .assign(key=1)
      .merge(dates_df.assign(key=1), on='key')
      .drop('key', axis=1))

# 4) replace NaT (i.e. “never happened by cutoff”) with far-future
for col in ['ia_date','wd_date','on_date']:
    projects[col] = projects[col].fillna(pd.Timestamp.max)

# 5) filter to only those still pending **at** each snapshot date
mask = (
    (projects['date'] >= projects['q_date']) &
    (projects['date'] <  projects['ia_date']) &
    (projects['date'] <  projects['wd_date']) &
    (projects['date'] <  projects['on_date']))
pending = projects.loc[mask, ['region','date','q_date']]

# 6) compute days_pending and then the daily average by region
pending['days_pending'] = (pending['date'] - pending['q_date']).dt.days

avg_pending = (
    pending
      .groupby(['region','date'])['days_pending']
      .mean()
      .reset_index(name='avg_days_pending'))

# d) merge with rolled
agg = (
    final
    .merge(avg_pending, on=['region','date'], how='right')
    .sort_values(['region','date']))

# final now has, for each ISO and each calendar date:
#   - count_ia, count_wd, count_on (30-day sums)
#   - avg_days_to_ia, avg_days_to_wd, avg_days_to_on (30-day means)
#   - avg_days_pending (daily average for all still-pending in that snapshot)

lmp = pd.read_csv('iso_data/yearly_combined_data/combined_ne_2023_lmp_data.csv')

# 1) Make sure your queue‐metrics DataFrame has a proper datetime “date” column
agg['date'] = pd.to_datetime(agg['date']).dt.normalize()
agg['date'] = pd.to_datetime(agg['date']).dt.tz_localize('UTC')

agg = agg.rename(columns={'region':'iso'})
agg = agg.drop(['count_ia', 'count_on', 'count_wd'], axis=1)

# 2) Prepare your LMP DataFrame
#    – parse timestamp_utc → datetime
#    – extract the date (drop the time component)
lmp['date'] = pd.to_datetime(lmp['timestamp_utc']).dt.normalize()

# 4) Merge the two tables on region & date
merged = (
    lmp
      .merge(agg, on=['iso','date'], how='left')
).drop(['date'], axis=1)

In [ ]:
# add in eia wholesale data

data_2022 = pd.read_excel("iso_data/Wholesale_Pricing_Data/ice_electric-2022final.xlsx")
data_2023 = pd.read_excel("iso_data/Wholesale_Pricing_Data/ice_electric-2023final.xlsx")
data_2024 = pd.read_excel("iso_data/Wholesale_Pricing_Data/ice_electric-2024final.xlsx")

eia_pricing_data = pd.concat([data_2022, data_2023, data_2024], ignore_index=True, axis=0)

hub_to_iso = {
    'Indiana Hub RT Peak': 'MISO',
    'Mid C Peak': 'Non-ISO (Mid-Columbia)',
    'NP15 EZ Gen DA LMP Peak': 'CAISO',
    'Nepool MH DA LMP Peak': 'ISO-NE',
    'PJM WH Real Time Peak': 'PJM',
    'Palo Verde Peak': 'CAISO',
    'SP15 EZ Gen DA LMP Peak': 'CAISO'
}

eia_pricing_data['ISO'] = eia_pricing_data['Price hub'].map(hub_to_iso)

# Convert dates to datetime
trade_dates = pd.to_datetime(eia_pricing_data['Trade date'], format='mixed', errors='coerce')
delivery_start_dates = pd.to_datetime(eia_pricing_data['Delivery start date'], format='mixed', errors='coerce')
delivery_end_dates = pd.to_datetime(eia_pricing_data['Delivery \nend date'], format='mixed', errors='coerce')

# Check the unique years
print("Trade date years:", sorted(trade_dates.dropna().dt.year.unique()))
print("Delivery start date years:", sorted(delivery_start_dates.dropna().dt.year.unique()))
print("Delivery end date years:", sorted(delivery_end_dates.dropna().dt.year.unique()))

eia_daily = eia_pricing_data.copy()

# Convert 'Trade date' column to datetime objects 
eia_daily['Trade date'] = pd.to_datetime(eia_daily['Trade date'], format='mixed').dt.date

# Create a new column for weighted price = price × volume
eia_daily['weighted_price'] = eia_daily['Wtd avg price $/MWh'] * eia_daily['Daily volume MWh']

# Group data by Trade date and ISO, and aggregate:
eia_daily_summary = (
    eia_daily
    .groupby(['Trade date', 'ISO'])
    .agg(
        weighted_avg_price=('weighted_price', 'sum'),           # sum of (P × V)
        total_volume=('Daily volume MWh', 'sum'),                # sum of volume
        total_trades=('Number of trades', 'sum'),                # sum of trades
        total_counterparties=('Number of counterparties', 'sum') # sum of counterparties
    )
    .reset_index()
)

# Calculate the volume-weighted average price
eia_daily_summary['Wtd avg price $/MWh'] = eia_daily_summary['weighted_avg_price'] / eia_daily_summary['total_volume']

# Select and reorder the final columns
eia_daily_summary = eia_daily_summary[
    ['Trade date', 'ISO', 'Wtd avg price $/MWh', 'total_volume', 'total_trades', 'total_counterparties']
]

# Filter to include only Trade dates from 2022, 2023, or 2024
eia_daily_summary = eia_daily_summary[
    eia_daily_summary['Trade date'].apply(lambda x: x.year).isin([2022, 2023, 2024])
].reset_index(drop=True)

merged['date'] = pd.to_datetime(temp_df['timestamp_utc']).dt.date

merged_df = merged.merge(
    eia_daily_summary,
    how='left',
    left_on=['date', 'iso'],
    right_on=['Trade date', 'ISO']
)

merged_df = merged_df.drop(columns=['Trade date', 'ISO', 'date', 'Unnamed: 0'])

merged_df.to_csv('iso_data/yearly_combined_data/final_ne_2023_df.csv')

#### MISO, ERCOT, SPP + Respective Interconnection Queues and Wholesale Electricity Trades

In [ ]:
combined_mw_south_raw_df = pd.concat([
    miso_2023_df,
    ercot_2023_df,
    spp_2023_df
], ignore_index=True)

print(combined_mw_south_raw_df.head())

print(f"✅ Combined raw DataFrame shape: {combined_mw_south_raw_df.shape}")

def clean_spp(df_spp):
    df = df_spp.rename(columns={
        'GMTIntervalEnd': 'timestamp_utc',
        'Pnode': 'Location Name',
    })
    df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
    df['timestamp_utc'] = df['timestamp_utc'].dt.tz_localize('UTC')
    df['Location Type'] = 'Node'
    df['iso'] = 'SPP'
    return df[['timestamp_utc', 'Location Name', 'Location Type', 'iso', 'LMP', 'MCC', 'MLC']]

def clean_ercot(df_ercot):
    df = df_ercot.copy()

    # Parse deliverydate to datetime
    df['deliverydate'] = pd.to_datetime(df['deliverydate'])

    # Fix 24:00 by shifting the date and setting hour to 0
    mask_24 = df['hourending'] == '24:00'
    df.loc[mask_24, 'deliverydate'] += pd.Timedelta(days=1)
    df.loc[mask_24, 'hourending'] = '00:00'

    # Now safely extract the hour as an integer
    df['hour'] = df['hourending'].str.extract('(\d+)').astype(int)

    
    # Step 1: Combine deliverydate and hourending as strings
    datetime_str = df['deliverydate'].astype(str) + ' ' + df['hourending'].astype(str)

    # Step 2: Parse the combined string into a real datetime
    df['timestamp_utc'] = pd.to_datetime(datetime_str, format='%Y-%m-%d %H:%M', errors='coerce')

    # Step 3: Localize to UTC
    df['timestamp_utc'] = df['timestamp_utc'].dt.tz_localize('UTC')


    # Standardize columns
    df['Location Name'] = df['busname']
    df['Location Type'] = 'Node'
    df['iso'] = 'ERCOT'
    df['MCC'] = None
    df['MLC'] = None
    df = df.rename(columns={'lmp': 'LMP'})
    
    return df[['timestamp_utc', 'Location Name', 'Location Type', 'iso', 'LMP', 'MCC', 'MLC']]


def reshape_miso(df_miso):
    # Identify the hour columns (HE1, HE2, ..., HE24)
    hour_columns = [col for col in df_miso.columns if str(col).startswith('HE')]

    # Melt from wide to long
    df_long = df_miso.melt(
        id_vars=['MARKET_DAY', 'NODE', 'TYPE', 'VALUE'],
        value_vars=hour_columns,
        var_name='Hour Ending',
        value_name='Price'
    )

    # ✅ Safe handling of 'Hour Ending'
    df_long['Hour Ending'] = df_long['Hour Ending'].astype(str)  # Ensure it's string
    df_long['Hour Ending'] = df_long['Hour Ending'].str.replace('HE', '', regex=False)
    df_long = df_long[df_long['Hour Ending'].str.isnumeric()]
    df_long['Hour Ending'] = df_long['Hour Ending'].astype(int)

    # Create local timestamp (interval beginning, so subtract 1 hour)
    df_long['Date'] = pd.to_datetime(df_long['MARKET_DAY'], format='%m/%d/%Y', errors='coerce')
    df_long['timestamp_local'] = df_long['Date'] + pd.to_timedelta(df_long['Hour Ending'] - 1, unit='h')

    # ✅ Handle DST properly
    df_long['timestamp_utc'] = (
        df_long['timestamp_local']
        .dt.tz_localize('US/Central', ambiguous='NaT', nonexistent='shift_forward')
        .dt.tz_convert('UTC')
    )

    # Rename columns to match final schema
    df_long = df_long.rename(columns={
        'NODE': 'Location Name',
        'TYPE': 'Location Type',
        'VALUE': 'Component'
    })

    df_long['iso'] = 'MISO'

    return df_long[['timestamp_utc', 'Location Name', 'Location Type', 'Component', 'Price', 'iso']]

def pivot_components(df_long):
    """
    Pivots 'Component' rows (LMP, MCC, MLC) into separate columns,
    keeping timestamp, location info, and ISO.
    """
    df_wide = df_long.pivot_table(
        index=['timestamp_utc', 'Location Name', 'Location Type', 'iso'],
        columns='Component',
        values='Price',
        aggfunc='first'   # ✅ Critical: Avoid aggregation crash on objects
    ).reset_index()

    df_wide.columns.name = None  # Remove pivot artifacts
    return df_wide



def combine_all(df_spp, df_ercot, df_miso):
    spp_clean = clean_spp(df_spp)
    ercot_clean = clean_ercot(df_ercot)
    df_miso_long = reshape_miso(df_miso)
    miso_clean = pivot_components(df_miso_long)
    combined = pd.concat([spp_clean, ercot_clean, miso_clean], ignore_index=True)
    return combined

combined_df = combine_all(spp_2023_df, ercot_2023_df, miso_2023_df)

combined_df = combined_df.sort_values('timestamp_utc').reset_index(drop=True)

# 🧠 Now set timestamp_utc as index
combined_df = combined_df.set_index('timestamp_utc')

combined_mw_south_df_2023 = combined_df[['iso', 'Location Name', 'Location Type', 'LMP', 'MCC', 'MLC']]
# add column to call everything a node (but keep location type as what ISO calls it
combined_mw_south_df_2023['node'] = 'node'
combined_mw_south_df_2023

output_path = 'iso_data/yearly_combined_data/combined_mw_south_2023_lmp_data.csv'
combined_mw_south_df_2023.to_csv(output_path)

print(f"✅ Successfully saved to {output_path}")

In [ ]:
# add in interconnection queue data
queues = pd.read_excel('iso_data/queues_2024.xlsx', sheet_name=1)

def standardize_dates(series):
    return pd.to_datetime(series, 
                          infer_datetime_format=True,
                          errors='coerce')

queues['q_date'] = standardize_dates(queues['q_date'])
queues['ia_date'] = standardize_dates(queues['ia_date'])
queues['wd_date'] = standardize_dates(queues['wd_date'])
queues['on_date'] = standardize_dates(queues['on_date'])

cutoff = pd.Timestamp('2022-01-01')
filtered_queues = queues.loc[(queues['q_date'] >= cutoff) & 
                             ((queues['ia_date'] >= cutoff)|(queues['ia_date'].isnull())) & 
                             ((queues['wd_date'] >= cutoff)|(queues['wd_date'].isnull())) &
                             ((queues['on_date'] >= cutoff)|(queues['on_date'].isnull()))]

# q_date: date when project entered queue
# ia_date: date of signed interconnection agreement
# wd_date: date project withdrawn from queue
# on_date: date project became operational

selected_cols = ['q_id', 'region', 'q_date', 
                 'ia_date', 'wd_date', 'on_date']

selected_regions = ['PJM', 'ERCOT', 'CAISO', 'SPP', 'NYISO', 'ISO-NE']

filtered_queues = filtered_queues[filtered_queues['region'].isin(selected_regions)].reset_index(drop=True)

def queue_duration(df, start, end):
    difference = df[end] - df[start]
    return difference.dt.days

filtered_queues['days_to_ia'] = queue_duration(filtered_queues, 'q_date', 'ia_date')
filtered_queues['days_to_wd'] = queue_duration(filtered_queues, 'q_date', 'wd_date')
filtered_queues['days_to_on'] = queue_duration(filtered_queues, 'q_date', 'on_date')

as_of = pd.to_datetime('2023-12-31')
pending = filtered_queues[['ia_date','wd_date','on_date']].isna().all(axis=1)

filtered_queues['days_pending'] = np.where(
    pending,
    (as_of - filtered_queues['q_date']).dt.days,
    np.nan)

selected_cols = selected_cols + ['days_to_ia', 'days_to_wd', 'days_to_on', 'days_pending']

# 1) build a “long” events frame: one row per event occurrence
events = []
for event, date_col, days_col in [
    ('ia', 'ia_date', 'days_to_ia'),
    ('wd', 'wd_date', 'days_to_wd'),
    ('on', 'on_date', 'days_to_on'),
]:
    tmp = (
        filtered_queues
        .loc[filtered_queues[date_col].notna(), ['region', date_col, days_col]]
        .rename(columns={date_col: 'date', days_col: 'days_to'})
    )
    tmp['event'] = event
    events.append(tmp)
events = pd.concat(events, ignore_index=True)

events.sort_values(by=['region','date'])

# 1) pivot out daily counts and daily average days_to
daily_counts = (
    events
      .groupby(['region','date','event'])
      .size()
      .unstack('event', fill_value=0)
      .rename(columns={'ia':'count_ia','wd':'count_wd','on':'count_on'}))

daily_days = (
    events
      .groupby(['region','date','event'])['days_to']
      .mean()
      .unstack('event')
      .rename(columns={'ia':'days_to_ia','wd':'days_to_wd','on':'days_to_on'}))

daily = (
    daily_counts
      .join(daily_days, how='outer')
      .sort_index())

# 2) reindex to every calendar date so cumsum/expanding works
all_dates = pd.date_range(
    events['date'].min().floor('D'),
    events['date'].max().ceil('D'),
    freq='D'
)
idx = pd.MultiIndex.from_product(
    [daily.index.levels[0], all_dates],
    names=['region','date']
)
daily = daily.reindex(idx, fill_value=0).sort_index()

# 3) cumulative sums of counts
daily['cum_ia'] = daily.groupby(level='region')['count_ia'].cumsum()
daily['cum_wd'] = daily.groupby(level='region')['count_wd'].cumsum()
daily['cum_on'] = daily.groupby(level='region')['count_on'].cumsum()

# 4) cumulative sums & counts of days_to, then expanding mean
for ev in ['ia','wd','on']:
    # running sum of days_to
    daily[f'sum_days_to_{ev}'] = (
        daily[f'days_to_{ev}']
          .groupby(level='region')
          .cumsum()
    )
    # running count of events (same as cum_count)
    daily[f'cnt_days_to_{ev}'] = daily[f'count_{ev}'].groupby(level='region').cumsum()
    # expanding average = sum / count
    daily[f'avg_days_to_{ev}'] = (
        daily[f'sum_days_to_{ev}'] / daily[f'cnt_days_to_{ev}'])

# 5) clean up: drop intermediate columns
final = daily.reset_index().drop(
    columns=[f'days_to_{ev}'      for ev in ['ia','wd','on']]
           +[f'sum_days_to_{ev}'  for ev in ['ia','wd','on']]
           +[f'cnt_days_to_{ev}'  for ev in ['ia','wd','on']])

# 0) your cutoff
as_of = pd.to_datetime('2023-12-31')

# 1) build the calendar of snapshot dates **only** through the cutoff
all_dates = pd.date_range(
    start=filtered_queues['q_date'].min().floor('D'),
    end=as_of,
    freq='D')

# 2) pull in each project’s dates
proj_raw = filtered_queues[['region','q_date','ia_date','wd_date','on_date']].copy()

# 3) Cartesian-merge so each project is paired with each snapshot date
dates_df = pd.DataFrame({'date': all_dates})
projects = (
    proj_raw
      .assign(key=1)
      .merge(dates_df.assign(key=1), on='key')
      .drop('key', axis=1))

# 4) replace NaT (i.e. “never happened by cutoff”) with far-future
for col in ['ia_date','wd_date','on_date']:
    projects[col] = projects[col].fillna(pd.Timestamp.max)

# 5) filter to only those still pending **at** each snapshot date
mask = (
    (projects['date'] >= projects['q_date']) &
    (projects['date'] <  projects['ia_date']) &
    (projects['date'] <  projects['wd_date']) &
    (projects['date'] <  projects['on_date']))
pending = projects.loc[mask, ['region','date','q_date']]

# 6) compute days_pending and then the daily average by region
pending['days_pending'] = (pending['date'] - pending['q_date']).dt.days

avg_pending = (
    pending
      .groupby(['region','date'])['days_pending']
      .mean()
      .reset_index(name='avg_days_pending'))

# d) merge with rolled
agg = (
    final
    .merge(avg_pending, on=['region','date'], how='right')
    .sort_values(['region','date']))

# final now has, for each ISO and each calendar date:
#   - count_ia, count_wd, count_on (30-day sums)
#   - avg_days_to_ia, avg_days_to_wd, avg_days_to_on (30-day means)
#   - avg_days_pending (daily average for all still-pending in that snapshot)

lmp = pd.read_csv('iso_data/yearly_combined_data/combined_mw_south_2023_lmp_data.csv')

# 1) Make sure your queue‐metrics DataFrame has a proper datetime “date” column
agg['date'] = pd.to_datetime(agg['date']).dt.normalize()
agg['date'] = pd.to_datetime(agg['date']).dt.tz_localize('UTC')

agg = agg.rename(columns={'region':'iso'})
agg = agg.drop(['count_ia', 'count_on', 'count_wd'], axis=1)

# 2) Prepare your LMP DataFrame
#    – parse timestamp_utc → datetime
#    – extract the date (drop the time component)
lmp['date'] = pd.to_datetime(lmp['timestamp_utc']).dt.normalize()

# 4) Merge the two tables on region & date
merged = (
    lmp
      .merge(agg, on=['iso','date'], how='left')
).drop(['date'], axis=1)

In [ ]:
# add in eia wholesale data

data_2022 = pd.read_excel("iso_data/Wholesale_Pricing_Data/ice_electric-2022final.xlsx")
data_2023 = pd.read_excel("iso_data/Wholesale_Pricing_Data/ice_electric-2023final.xlsx")
data_2024 = pd.read_excel("iso_data/Wholesale_Pricing_Data/ice_electric-2024final.xlsx")

eia_pricing_data = pd.concat([data_2022, data_2023, data_2024], ignore_index=True, axis=0)

hub_to_iso = {
    'Indiana Hub RT Peak': 'MISO',
    'Mid C Peak': 'Non-ISO (Mid-Columbia)',
    'NP15 EZ Gen DA LMP Peak': 'CAISO',
    'Nepool MH DA LMP Peak': 'ISO-NE',
    'PJM WH Real Time Peak': 'PJM',
    'Palo Verde Peak': 'CAISO',
    'SP15 EZ Gen DA LMP Peak': 'CAISO'
}

eia_pricing_data['ISO'] = eia_pricing_data['Price hub'].map(hub_to_iso)

# Convert dates to datetime
trade_dates = pd.to_datetime(eia_pricing_data['Trade date'], format='mixed', errors='coerce')
delivery_start_dates = pd.to_datetime(eia_pricing_data['Delivery start date'], format='mixed', errors='coerce')
delivery_end_dates = pd.to_datetime(eia_pricing_data['Delivery \nend date'], format='mixed', errors='coerce')

# Check the unique years
print("Trade date years:", sorted(trade_dates.dropna().dt.year.unique()))
print("Delivery start date years:", sorted(delivery_start_dates.dropna().dt.year.unique()))
print("Delivery end date years:", sorted(delivery_end_dates.dropna().dt.year.unique()))

eia_daily = eia_pricing_data.copy()

# Convert 'Trade date' column to datetime objects 
eia_daily['Trade date'] = pd.to_datetime(eia_daily['Trade date'], format='mixed').dt.date

# Create a new column for weighted price = price × volume
eia_daily['weighted_price'] = eia_daily['Wtd avg price $/MWh'] * eia_daily['Daily volume MWh']

# Group data by Trade date and ISO, and aggregate:
eia_daily_summary = (
    eia_daily
    .groupby(['Trade date', 'ISO'])
    .agg(
        weighted_avg_price=('weighted_price', 'sum'),           # sum of (P × V)
        total_volume=('Daily volume MWh', 'sum'),                # sum of volume
        total_trades=('Number of trades', 'sum'),                # sum of trades
        total_counterparties=('Number of counterparties', 'sum') # sum of counterparties
    )
    .reset_index()
)

# Calculate the volume-weighted average price
eia_daily_summary['Wtd avg price $/MWh'] = eia_daily_summary['weighted_avg_price'] / eia_daily_summary['total_volume']

# Select and reorder the final columns
eia_daily_summary = eia_daily_summary[
    ['Trade date', 'ISO', 'Wtd avg price $/MWh', 'total_volume', 'total_trades', 'total_counterparties']
]

# Filter to include only Trade dates from 2022, 2023, or 2024
eia_daily_summary = eia_daily_summary[
    eia_daily_summary['Trade date'].apply(lambda x: x.year).isin([2022, 2023, 2024])
].reset_index(drop=True)

merged['date'] = pd.to_datetime(temp_df['timestamp_utc']).dt.date

merged_df = merged.merge(
    eia_daily_summary,
    how='left',
    left_on=['date', 'iso'],
    right_on=['Trade date', 'ISO']
)

merged_df = merged_df.drop(columns=['Trade date', 'ISO', 'date', 'Unnamed: 0'])

merged_df.to_csv('iso_data/yearly_combined_data/final_mf_south_2023_df.csv')

#### CAISO Respective Interconnection Queues and Wholesale Electricity Trades

In [ ]:
caiso = pd.read_csv('iso_data/caiso_data/caiso_dam_lmp_parallel.csv')
caiso

df_new = pd.DataFrame()

df_new['timestamp_utc'] = pd.to_datetime(caiso['INTERVALENDTIME_GMT'], errors='coerce')
df_new['iso'] = 'CAISO'
df_new['Location Name'] = caiso['NODE']
df_new['Location Type'] = 'Node'
df_new['LMP'] = caiso['MW']
df_new['MCC'] = pd.NA  
df_new['MLC'] = pd.NA
df_new['node'] = 'node'

df_new = df_new.sort_values('timestamp_utc').reset_index(drop=True)

df_new = df_new.set_index('timestamp_utc')
df_new

output_path = 'iso_data/yearly_combined_data/caiso_2023_lmp_data.csv'
df_new.to_csv(output_path)

In [ ]:
# add in interconnection queue data
queues = pd.read_excel('iso_data/queues_2024.xlsx', sheet_name=1)

def standardize_dates(series):
    return pd.to_datetime(series, 
                          infer_datetime_format=True,
                          errors='coerce')

queues['q_date'] = standardize_dates(queues['q_date'])
queues['ia_date'] = standardize_dates(queues['ia_date'])
queues['wd_date'] = standardize_dates(queues['wd_date'])
queues['on_date'] = standardize_dates(queues['on_date'])

cutoff = pd.Timestamp('2022-01-01')
filtered_queues = queues.loc[(queues['q_date'] >= cutoff) & 
                             ((queues['ia_date'] >= cutoff)|(queues['ia_date'].isnull())) & 
                             ((queues['wd_date'] >= cutoff)|(queues['wd_date'].isnull())) &
                             ((queues['on_date'] >= cutoff)|(queues['on_date'].isnull()))]

# q_date: date when project entered queue
# ia_date: date of signed interconnection agreement
# wd_date: date project withdrawn from queue
# on_date: date project became operational

selected_cols = ['q_id', 'region', 'q_date', 
                 'ia_date', 'wd_date', 'on_date']

selected_regions = ['PJM', 'ERCOT', 'CAISO', 'SPP', 'NYISO', 'ISO-NE']

filtered_queues = filtered_queues[filtered_queues['region'].isin(selected_regions)].reset_index(drop=True)

def queue_duration(df, start, end):
    difference = df[end] - df[start]
    return difference.dt.days

filtered_queues['days_to_ia'] = queue_duration(filtered_queues, 'q_date', 'ia_date')
filtered_queues['days_to_wd'] = queue_duration(filtered_queues, 'q_date', 'wd_date')
filtered_queues['days_to_on'] = queue_duration(filtered_queues, 'q_date', 'on_date')

as_of = pd.to_datetime('2023-12-31')
pending = filtered_queues[['ia_date','wd_date','on_date']].isna().all(axis=1)

filtered_queues['days_pending'] = np.where(
    pending,
    (as_of - filtered_queues['q_date']).dt.days,
    np.nan)

selected_cols = selected_cols + ['days_to_ia', 'days_to_wd', 'days_to_on', 'days_pending']

# 1) build a “long” events frame: one row per event occurrence
events = []
for event, date_col, days_col in [
    ('ia', 'ia_date', 'days_to_ia'),
    ('wd', 'wd_date', 'days_to_wd'),
    ('on', 'on_date', 'days_to_on'),
]:
    tmp = (
        filtered_queues
        .loc[filtered_queues[date_col].notna(), ['region', date_col, days_col]]
        .rename(columns={date_col: 'date', days_col: 'days_to'})
    )
    tmp['event'] = event
    events.append(tmp)
events = pd.concat(events, ignore_index=True)

events.sort_values(by=['region','date'])

# 1) pivot out daily counts and daily average days_to
daily_counts = (
    events
      .groupby(['region','date','event'])
      .size()
      .unstack('event', fill_value=0)
      .rename(columns={'ia':'count_ia','wd':'count_wd','on':'count_on'}))

daily_days = (
    events
      .groupby(['region','date','event'])['days_to']
      .mean()
      .unstack('event')
      .rename(columns={'ia':'days_to_ia','wd':'days_to_wd','on':'days_to_on'}))

daily = (
    daily_counts
      .join(daily_days, how='outer')
      .sort_index())

# 2) reindex to every calendar date so cumsum/expanding works
all_dates = pd.date_range(
    events['date'].min().floor('D'),
    events['date'].max().ceil('D'),
    freq='D'
)
idx = pd.MultiIndex.from_product(
    [daily.index.levels[0], all_dates],
    names=['region','date']
)
daily = daily.reindex(idx, fill_value=0).sort_index()

# 3) cumulative sums of counts
daily['cum_ia'] = daily.groupby(level='region')['count_ia'].cumsum()
daily['cum_wd'] = daily.groupby(level='region')['count_wd'].cumsum()
daily['cum_on'] = daily.groupby(level='region')['count_on'].cumsum()

# 4) cumulative sums & counts of days_to, then expanding mean
for ev in ['ia','wd','on']:
    # running sum of days_to
    daily[f'sum_days_to_{ev}'] = (
        daily[f'days_to_{ev}']
          .groupby(level='region')
          .cumsum()
    )
    # running count of events (same as cum_count)
    daily[f'cnt_days_to_{ev}'] = daily[f'count_{ev}'].groupby(level='region').cumsum()
    # expanding average = sum / count
    daily[f'avg_days_to_{ev}'] = (
        daily[f'sum_days_to_{ev}'] / daily[f'cnt_days_to_{ev}'])

# 5) clean up: drop intermediate columns
final = daily.reset_index().drop(
    columns=[f'days_to_{ev}'      for ev in ['ia','wd','on']]
           +[f'sum_days_to_{ev}'  for ev in ['ia','wd','on']]
           +[f'cnt_days_to_{ev}'  for ev in ['ia','wd','on']])

# 0) your cutoff
as_of = pd.to_datetime('2023-12-31')

# 1) build the calendar of snapshot dates **only** through the cutoff
all_dates = pd.date_range(
    start=filtered_queues['q_date'].min().floor('D'),
    end=as_of,
    freq='D')

# 2) pull in each project’s dates
proj_raw = filtered_queues[['region','q_date','ia_date','wd_date','on_date']].copy()

# 3) Cartesian-merge so each project is paired with each snapshot date
dates_df = pd.DataFrame({'date': all_dates})
projects = (
    proj_raw
      .assign(key=1)
      .merge(dates_df.assign(key=1), on='key')
      .drop('key', axis=1))

# 4) replace NaT (i.e. “never happened by cutoff”) with far-future
for col in ['ia_date','wd_date','on_date']:
    projects[col] = projects[col].fillna(pd.Timestamp.max)

# 5) filter to only those still pending **at** each snapshot date
mask = (
    (projects['date'] >= projects['q_date']) &
    (projects['date'] <  projects['ia_date']) &
    (projects['date'] <  projects['wd_date']) &
    (projects['date'] <  projects['on_date']))
pending = projects.loc[mask, ['region','date','q_date']]

# 6) compute days_pending and then the daily average by region
pending['days_pending'] = (pending['date'] - pending['q_date']).dt.days

avg_pending = (
    pending
      .groupby(['region','date'])['days_pending']
      .mean()
      .reset_index(name='avg_days_pending'))

# d) merge with rolled
agg = (
    final
    .merge(avg_pending, on=['region','date'], how='right')
    .sort_values(['region','date']))

# final now has, for each ISO and each calendar date:
#   - count_ia, count_wd, count_on (30-day sums)
#   - avg_days_to_ia, avg_days_to_wd, avg_days_to_on (30-day means)
#   - avg_days_pending (daily average for all still-pending in that snapshot)

lmp = pd.read_csv('iso_data/yearly_combined_data/caiso_2023_lmp_data.csv')

# 1) Make sure your queue‐metrics DataFrame has a proper datetime “date” column
agg['date'] = pd.to_datetime(agg['date']).dt.normalize()
agg['date'] = pd.to_datetime(agg['date']).dt.tz_localize('UTC')

agg = agg.rename(columns={'region':'iso'})
agg = agg.drop(['count_ia', 'count_on', 'count_wd'], axis=1)

# 2) Prepare your LMP DataFrame
#    – parse timestamp_utc → datetime
#    – extract the date (drop the time component)
lmp['date'] = pd.to_datetime(lmp['timestamp_utc']).dt.normalize()

# 4) Merge the two tables on region & date
merged = (
    lmp
      .merge(agg, on=['iso','date'], how='left')
).drop(['date'], axis=1)

In [ ]:
# add in eia wholesale data

data_2022 = pd.read_excel("iso_data/Wholesale_Pricing_Data/ice_electric-2022final.xlsx")
data_2023 = pd.read_excel("iso_data/Wholesale_Pricing_Data/ice_electric-2023final.xlsx")
data_2024 = pd.read_excel("iso_data/Wholesale_Pricing_Data/ice_electric-2024final.xlsx")

eia_pricing_data = pd.concat([data_2022, data_2023, data_2024], ignore_index=True, axis=0)

hub_to_iso = {
    'Indiana Hub RT Peak': 'MISO',
    'Mid C Peak': 'Non-ISO (Mid-Columbia)',
    'NP15 EZ Gen DA LMP Peak': 'CAISO',
    'Nepool MH DA LMP Peak': 'ISO-NE',
    'PJM WH Real Time Peak': 'PJM',
    'Palo Verde Peak': 'CAISO',
    'SP15 EZ Gen DA LMP Peak': 'CAISO'
}

eia_pricing_data['ISO'] = eia_pricing_data['Price hub'].map(hub_to_iso)

# Convert dates to datetime
trade_dates = pd.to_datetime(eia_pricing_data['Trade date'], format='mixed', errors='coerce')
delivery_start_dates = pd.to_datetime(eia_pricing_data['Delivery start date'], format='mixed', errors='coerce')
delivery_end_dates = pd.to_datetime(eia_pricing_data['Delivery \nend date'], format='mixed', errors='coerce')

# Check the unique years
print("Trade date years:", sorted(trade_dates.dropna().dt.year.unique()))
print("Delivery start date years:", sorted(delivery_start_dates.dropna().dt.year.unique()))
print("Delivery end date years:", sorted(delivery_end_dates.dropna().dt.year.unique()))

eia_daily = eia_pricing_data.copy()

# Convert 'Trade date' column to datetime objects 
eia_daily['Trade date'] = pd.to_datetime(eia_daily['Trade date'], format='mixed').dt.date

# Create a new column for weighted price = price × volume
eia_daily['weighted_price'] = eia_daily['Wtd avg price $/MWh'] * eia_daily['Daily volume MWh']

# Group data by Trade date and ISO, and aggregate:
eia_daily_summary = (
    eia_daily
    .groupby(['Trade date', 'ISO'])
    .agg(
        weighted_avg_price=('weighted_price', 'sum'),           # sum of (P × V)
        total_volume=('Daily volume MWh', 'sum'),                # sum of volume
        total_trades=('Number of trades', 'sum'),                # sum of trades
        total_counterparties=('Number of counterparties', 'sum') # sum of counterparties
    )
    .reset_index()
)

# Calculate the volume-weighted average price
eia_daily_summary['Wtd avg price $/MWh'] = eia_daily_summary['weighted_avg_price'] / eia_daily_summary['total_volume']

# Select and reorder the final columns
eia_daily_summary = eia_daily_summary[
    ['Trade date', 'ISO', 'Wtd avg price $/MWh', 'total_volume', 'total_trades', 'total_counterparties']
]

# Filter to include only Trade dates from 2022, 2023, or 2024
eia_daily_summary = eia_daily_summary[
    eia_daily_summary['Trade date'].apply(lambda x: x.year).isin([2022, 2023, 2024])
].reset_index(drop=True)

merged['date'] = pd.to_datetime(temp_df['timestamp_utc']).dt.date

merged_df = merged.merge(
    eia_daily_summary,
    how='left',
    left_on=['date', 'iso'],
    right_on=['Trade date', 'ISO']
)

merged_df = merged_df.drop(columns=['Trade date', 'ISO', 'date', 'Unnamed: 0'])

merged_df.to_csv('iso_data/yearly_combined_data/final_caiso_2023_df.csv')